In [5]:
root_dir = 'C:\\Users\\baekj\\Downloads\\dog_image'

In [6]:
import os
import shutil

if os.path.exists( os.path.join(root_dir, 'dog_image') ):
    shutil.rmtree( os.path.join(root_dir, 'dog_image') )

In [7]:
import os
import shutil

if not os.path.exists(os.path.join(root_dir, '\\test')):    
    os.mkdir(os.path.join(root_dir, '\\test'))

if not os.path.exists(os.path.join(root_dir, '\\test_image_files')):    
    os.mkdir(os.path.join(root_dir, '\\test_image_files'))

In [8]:
# 데이터 전체 개수 확인

import os
import glob

label_name_list = os.listdir(os.path.join(root_dir, 'C:\\Users\\baekj\\Downloads\\dog_image\\train'))

print('total label nums = ', len(label_name_list))
print('=================================================')
print(label_name_list)

total label nums =  4
['chihuahua', 'jindo_dog', 'shepherd', 'yorkshire_terrier']


In [9]:
import os
import glob
import shutil

ratio = 0.1    # train : test = 90 : 10

src_root_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/train')
dst_root_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/test')

label_name_list = os.listdir(src_root_dir) 

for label_name in label_name_list:   # test 디렉토리에 label 디렉토리 생성
    dst_label_name_dir = os.path.join(dst_root_dir, label_name)

    if not os.path.exists(dst_label_name_dir):
        os.mkdir(dst_label_name_dir)

#for label_name in label_name_list:    # 파일 move src dir => dst dir
#    train_image_file_list = glob.glob(os.path.join(src_root_dir, label_name, '*'))
#    split_num = int(ratio * len(train_image_file_list))
#    test_image_file_list = train_image_file_list[0:split_num]

#    for image_file in test_image_file_list:
#        shutil.move(image_file, os.path.join(dst_root_dir, label_name))    # move

In [28]:
# train : test 데이터 비율 확인

src_root_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/train')
dst_root_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/test')

train_label_name_list = os.listdir(src_root_dir)
test_label_name_list = os.listdir(dst_root_dir)

train_label_name_list.sort()
test_label_name_list.sort()

if train_label_name_list != test_label_name_list:
    print('fatal error !!!!')
else:
    print(len(train_label_name_list), len(test_label_name_list))

# 데이터 개수 확인

for label_name in train_label_name_list:

    train_data_nums = len(os.listdir(os.path.join(src_root_dir, label_name)))
    test_data_nums = len(os.listdir(os.path.join(dst_root_dir, label_name)))

    print('train => ', label_name, train_data_nums, ' , test => ', label_name, test_data_nums)
    print('=======================================================')

4 4
train =>  chihuahua 242  , test =>  chihuahua 123
train =>  jindo_dog 237  , test =>  jindo_dog 123
train =>  shepherd 249  , test =>  shepherd 127
train =>  yorkshire_terrier 251  , test =>  yorkshire_terrier 128


In [10]:
import os
import glob
import shutil

src_root_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/test/')
dst_root_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/test_image_files/')

# 대상 디렉토리가 없다면 생성합니다.
if not os.path.exists(dst_root_dir):
    os.makedirs(dst_root_dir)

label_name_list = os.listdir(src_root_dir)

for label_name in label_name_list:  # 파일 copy src dir => dst dir
    image_file_list = glob.glob(os.path.join(src_root_dir, label_name, '*'))
    print('total [%s] image file nums => [%s]' % (label_name, len(image_file_list)))

    copy_nums = 0

    for image_file in image_file_list:
        shutil.copy(image_file, os.path.join(dst_root_dir, os.path.basename(image_file)))  # copy 
        copy_nums = copy_nums + 1

    print('total copy nums => ', copy_nums)

total [chihuahua] image file nums => [123]
total copy nums =>  123
total [jindo_dog] image file nums => [123]
total copy nums =>  123
total [shepherd] image file nums => [127]
total copy nums =>  127
total [yorkshire_terrier] image file nums => [128]
total copy nums =>  128


In [11]:
IMG_WIDTH = 224  
IMG_HEIGHT = 224  


train_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/train/')
validation_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/train/')
test_dir = os.path.join(root_dir, 'C:/Users/baekj/Downloads/dog_image/test/')


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, 
                                    height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                    validation_split=0.15)

validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)


In [21]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, color_mode='rgb', 
                                                class_mode='sparse', subset = 'training',
                                                target_size=(IMG_WIDTH,IMG_HEIGHT))

validation_generator = validation_datagen.flow_from_directory(validation_dir, batch_size=16, color_mode='rgb', 
                                                          class_mode='sparse', subset = 'validation',
                                                          target_size=(IMG_WIDTH,IMG_HEIGHT))

Found 834 images belonging to 4 classes.
Found 145 images belonging to 4 classes.


In [15]:
print(train_generator.class_indices)

{'chihuahua': 0, 'jindo_dog': 1, 'shepherd': 2, 'yorkshire_terrier': 3}


In [16]:
import tensorflow as tf 
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
import scipy

In [17]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT,3))

In [18]:
model = Sequential()

model.add(base_model)

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))                  
model.add(Dense(4, activation='softmax'))

In [19]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(2e-5), metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 32)                1605664   
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 132       
                                                                 
Total params: 4834660 (18.44 MB)
Trainable params: 4812772 (18.36 MB)
Non-trainable params: 21888 (85.50 KB)
_____________

In [23]:
import scipy  # 올바른 대소문자로 임포트

from tensorflow.keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', patience=5)                     

hist = model.fit(train_generator, validation_data=validation_generator,
                 epochs=5, callbacks=[earlystopping])

Epoch 1/5
53/53 [==============================] - 169s 3s/step - loss: 0.8603 - accuracy: 0.1799 - val_loss: 0.3839 - val_accuracy: 0.2483
Epoch 2/5
53/53 [==============================] - 187s 4s/step - loss: 0.6310 - accuracy: 0.1787 - val_loss: 0.2734 - val_accuracy: 0.2414
Epoch 3/5
53/53 [==============================] - 162s 3s/step - loss: 0.5345 - accuracy: 0.1547 - val_loss: 0.2355 - val_accuracy: 0.2552
Epoch 4/5
53/53 [==============================] - 170s 3s/step - loss: 0.4403 - accuracy: 0.1811 - val_loss: 0.1653 - val_accuracy: 0.2345
Epoch 5/5
53/53 [==============================] - 150s 3s/step - loss: 0.3820 - accuracy: 0.1439 - val_loss: 0.1669 - val_accuracy: 0.2276
